In [2]:
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
import torch
import wandb

In [3]:
# CHECK DEVICE (CPU / GPU)
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Currently Using :: ", device)

Currently Using ::  cuda


In [4]:
!wandb login 57566fbb0e091de2e298a4320d872f9a2b200d12

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
def load_data(batch_size,img_size, augmentation):
    if augmentation == "Yes":
        train_augmentation = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    elif augmentation == "No":
        train_augmentation = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    test_augmentation = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    # Load the dataset
    data_path = "/kaggle/input/nature/inaturalist_12K"
    train_dataset = datasets.ImageFolder(os.path.join(data_path, 'train'), transform = train_augmentation)
    test_dataset = datasets.ImageFolder(os.path.join(data_path, 'val'), transform = test_augmentation)

    
    labels = train_dataset.classes
    trainset, valset = random_split(train_dataset, [8000, 1999])

    train_loader = DataLoader(trainset, batch_size = batch_size, num_workers = 2)
    val_loader = DataLoader(valset, batch_size = batch_size, num_workers = 2)
    test_loader = DataLoader(test_dataset, batch_size = batch_size, num_workers = 2)

    return labels , train_loader, val_loader, test_loader



In [7]:
from typing import Any


class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, PARAM) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.filter_org = PARAM["filter_org"]
        self.filter_num = PARAM["filter_num"]
        self.activation = PARAM["activation"]
        self.con_layers = PARAM["con_layers"]
        self.den_layers = PARAM["dense_layers"]
        self.input_channel = PARAM["input_channel"]
        self.filter_num_list = self.organize_filters(self.filter_org, self.filter_num, self.con_layers)
        self.filter_size_list = PARAM["filter_size"]
        self.act = self.activation_fun(PARAM["activation"])
        self.output_act = self.activation_fun(PARAM["output_activation"])
        self.padding = PARAM["padding"]
        self.stride = PARAM["stride"]
        self.pool_padding = PARAM["pool_padding"]
        self.pool_stride = PARAM["pool_stride"]
        self.dense_output_list = PARAM["dense_output_list"]
        self.image_size = PARAM["image_size"]
        self.pool_filter_size = PARAM["pool_filter_size"]
        self.dropout_list = PARAM["dropout"]
        self.batch_norm = PARAM["batch_norm"]
        self.create_con_layers(self.input_channel, self.filter_size_list, self.dense_output_list, self.filter_num_list, self.act, self.pool_filter_size, self.output_act, self.image_size, self.dropout_list)
        

    def create_con_layers(self, input_channel, filter_size_list, dense_output_list, filter_num_list, act, pool_filter_size, output_act, image_size, dropout_list):
        self.layers = nn.ModuleList()
        computation = 0

        #canvolution layer
        for i in range(1, self.con_layers+1):
            comp = 0
            layer = nn.Sequential(nn.Conv2d(input_channel, filter_num_list[i-1], filter_size_list[i-1], padding=self.padding, stride=self.stride), act, nn.MaxPool2d(pool_filter_size, padding=self.pool_padding, stride=self.pool_stride), nn.Dropout(dropout_list[i-1]))
            
#             for module in layer:
#                 if isinstance(module, nn.Conv2d):
#                     nn.init.xavier_uniform_(module.weight)

            image_size = (image_size - filter_size_list[i-1] + 2 * self.padding)//self.stride + 1
            
            comp = ((filter_size_list[i-1] ** 2) * input_channel * (image_size ** 2)*filter_num_list[i-1] + filter_num_list[i-1])
            computation += comp
            image_size = (image_size + 2 * self.pool_padding-(1*(pool_filter_size-1))-1)//self.pool_stride + 1
            print(image_size)
            # print(comp)
            input_channel = filter_num_list[i-1]
            self.layers.append(layer)
        dense_input = filter_num_list[self.con_layers-1] * (image_size ** 2)

        #dense layer 
        for i in range(1, self.den_layers+1):
            comp = 0
            layer = nn.Sequential(nn.Linear(dense_input, dense_output_list[i-1]), act, nn.Dropout(dropout_list[self.con_layers + i-1]))
            
#             for module in layer:
#                 if isinstance(module, nn.Conv2d):
#                     nn.init.xavier_uniform_(module.weight)

            comp = ((dense_input  + 1) * dense_output_list[i-1])
            computation += comp
            dense_input = dense_output_list[i-1]
            self.layers.append(layer)
            # print(computation)
            # print(comp)
        layer = nn.Sequential(nn.Linear(dense_input, 10))

#         for module in layer:
#                 if isinstance(module, nn.Conv2d):
#                     nn.init.xavier_uniform_(module.weight)

        comp = ((dense_input  + 1) * 10)
        computation += comp
        # print(comp)
        print("Computation :: ", computation)
        self.layers.append(layer)
        
        for layer in range(self.con_layers + self.den_layers + 1):
            if isinstance(self.layers[layer], nn.Conv2d):
                nn.init.xavier_uniform_(self.layers[layer].weight)
            if self.batch_norm == "Yes" and layer < self.con_layers:
                self.layers[layer].append(nn.BatchNorm2d(filter_num_list[layer]))


    def organize_filters(self, filter_org, filter_number, layers):
        if filter_org == "same":
            filter_num = [filter_number] * layers
        elif filter_org == "double":
            filter_num = [filter_number * (2 ** i) for i in range(layers)]
        elif filter_org == "half":
            filter_num = [int(filter_number * (2 ** (-i))) for i in range(layers)]
        return filter_num
    
    
    def activation_fun(self, act):
        if act == "ReLU":
            act_fun =nn.ReLU()
        elif act == "GELU":
            act_fun = nn.GELU()
        elif act == "SiLU":
            act_fun = nn.SiLU()
        elif act == "Mish":
            act_fun = nn.Mish()
        elif act == "softmax":
            act_fun = nn.Softmax(dim=1)
        elif act == "ELU":
            act_fun = nn.ELU()
        return act_fun
    
    
    def forward(self,x):
        for i in range(0, self.con_layers):
            x = self.layers[i](x)
        x = self.flatten(x)
        for i in range(0, self.den_layers):
            x = self.layers[i+self.con_layers](x)
        x = self.layers[self.con_layers + self.den_layers](x)
        return x   


In [8]:
PARAM = {
    "con_layers" : 5,
    "dense_layers" : 1,
    "filter_size" : [3] * 5,
    "output_activation" : "softmax", 
    "dense_output_list" : [16],
    "filter_num" : 8,
    "activation" : "ReLU",
    "filter_org" : "same", #double half
    "input_channel" : 3,
    "padding" : 0,
    "stride" : 1,
    "pool_padding" : 0,
    "pool_stride" : 1,
    "image_size" : 256,
    "pool_filter_size" : 3,
    "batch_size" : 64,
    "eta" : 0.0005,
    "dropout" : [0] * 6,
    "epochs" : 1,
    "augmentation" : "Yes",
    "batch_norm" : "Yes",
}

net = ConvolutionalNeuralNetwork(PARAM).to(device)

252
248
244
240
236
Computation ::  158281794


In [9]:
def train_model(model, device, PARAM):
    wandb.init(project='DL_Assignment2')
    wandb.run.name = 'SAMPLE-RUN'
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), PARAM["eta"])
    labels, train_loader, val_loader, test_loader = load_data(PARAM["batch_size"], PARAM["image_size"], PARAM["augmentation"])
    for epoch in range(PARAM["epochs"]):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        count = 0
        for images, labels in train_loader:
            # print("image is loaded")
            images = images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            # print(outputs)
            
            # Calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            # print(_, predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            count += 1
            print(count, end = " | ")
            if count%30 == 0:
                print()
        print("")
        model.eval()
        val_loss = 0.0
        correct_pred = 0
        total_pred = 0

        with torch.no_grad():
            for val_img, val_label in val_loader:
                val_img = val_img.to(device)
                val_label = val_label.to(device)
                val_output = model.forward(val_img)
                val = criterion(val_output, val_label)
                val_loss += val.item()

                idx, class_ = torch.max(val_output.data, 1)
                total_pred += val_label.size(0)
                correct_pred += (class_ == val_label).sum().item()

        
        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}, Training_Accuracy: {100 * correct / total}%, Validation_Loss : {val_loss/len(val_loader)}, Validation_accuracy : {100 * correct_pred / total_pred}%")

        wandb.log(
            {
                'epochs' : epoch+1,
                'training_loss' : running_loss/len(train_loader),
                'training_accuracy' : 100 * correct / total,
                'Validation_Loss' : val_loss/len(val_loader),
                'Validation_accuracy' : 100 * correct_pred / total_pred
            }
        )
        print("")
    return 100 * correct_pred / total_pred
        
        

In [8]:
# train_model(net, device, PARAM)

In [13]:
sweep_config = {
            'name': 'sweep_2 ',
            'method': 'grid',
            'metric': { 'goal': 'maximize','name': 'Accuracy'},
            'parameters': 
                {
                    'epochs' : {'values': [10]},
                    'batch_size' : {'values': [32, 64]},
                    'learning_rate' : {'values': [0.0001, 0.0005]},
                    'dense_sizes' : {'values': [16, 32]},
                    'activation' : {'values': ['ReLU', 'GELU', 'ELU', 'SiLU', 'Mish']},
                    'kernel' : {'values' : [8]},
                    'data_augmentation' : {'values' : ["Yes"]},
                    'kernel_aug' : {'values' : ["same"]},
                    'dropout' : {'values' : [0, 0.2]},
                    'batch_norm' : {'values' : ["Yes"]},
                    'filter_size' : {'values' : [3]}
                }
            }


In [14]:
def train():
    var1 = wandb.init(project="DL_Assignment2")
    var2 = var1.config

    # wandb.run.name = 'Optimizer:- ' + var2.optimizer + ' Epoch:- ' + str(var2.epochs) + " Avtivation_Function :- " + var2.activation + " Batch_Size :- " + str(var2.batch_size) + " Initialization :- " + var2.initialization + \
    #                 ' layers:-' + str(len(var2.hidden_layers)) +' decay:-' + str(var2.weight_decay) + ' beta:-' + str(var2.beta) + ' learning_rate:-' + str(var2.learning_rate) + \
    #                 ' beta2 :- ' + str(var2.beta)
    
    wandb.run.name = (f"epochs:{var2.epochs}_lr:{var2.learning_rate}_batch_size:{var2.batch_size}_filter_size:{var2.filter_size}_act:{var2.activation}_dropout:{var2.dropout}")

    PARAM = {
        "con_layers" : 5,
        "dense_layers" : 1,
        "filter_size" :  [var2.filter_size]* 5,
        "output_activation" : "softmax", 
        "dense_output_list" : [var2.dense_sizes],
        "filter_num" : var2.kernel,
        "activation" : var2.activation,
        "filter_org" : "same", #double half
        "input_channel" : 3,
        "padding" : 0,
        "stride" : 1,
        "pool_padding" : 0,
        "pool_stride" : 1,
        "image_size" : 256,
        "pool_filter_size" : 3,
        "batch_size" : var2.batch_size,
        "eta" : var2.learning_rate,
        "dropout" : [var2.dropout] * 6,
        "epochs" : var2.epochs,
        "augmentation" : var2.data_augmentation,
        "batch_norm" : var2.batch_norm
    }

    model = ConvolutionalNeuralNetwork(PARAM).to(device)
    accuracy = train_model(model, device, PARAM)
    wandb.log({
                "Accuracy" : accuracy
            })


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="DL_Assignment2")
wandb.agent(sweep_id, train)
wandb.finish()

Create sweep with ID: c5fywa1f
Sweep URL: https://wandb.ai/cs23m026/DL_Assignment2/sweeps/c5fywa1f


wandb: Agent Starting Run: f1xfaco0 with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: Yes
wandb: 	batch_size: 32
wandb: 	data_augmentation: Yes
wandb: 	dense_sizes: 16
wandb: 	dropout: 0
wandb: 	epochs: 10
wandb: 	filter_size: 3
wandb: 	kernel: 8
wandb: 	kernel_aug: same
wandb: 	learning_rate: 0.0001
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


252
248
244
240
236
Computation ::  158281794


1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 
31 | 32 | 33 | 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44 | 45 | 46 | 47 | 48 | 49 | 50 | 51 | 52 | 53 | 54 | 55 | 56 | 57 | 58 | 59 | 60 | 
61 | 62 | 63 | 64 | 65 | 66 | 67 | 68 | 69 | 70 | 71 | 72 | 73 | 74 | 75 | 76 | 77 | 78 | 79 | 80 | 81 | 82 | 83 | 84 | 85 | 86 | 87 | Epoch 1, Training Loss: 2.6592461948394774, Training_Accuracy: 11.25%, Validation_Loss : 2.567122845422654, Validation_accuracy : 11.505752876438219%

1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 
31 | 32 | 33 | 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44 | 45 | 46 | 47 | 48 | 49 | 50 | 51 | 52 | 53 | 54 | 55 | 56 | 57 | 58 | 59 | 60 | 
61 | 62 | 63 | 64 | 65 | 66 | 67 | 68 | 69 | 70 | 71 | 72 | 73 | 74 | 75 | 76 | 77 | 78 | 79 | 80 | 81 | 82 | 83 | 84 | 85 | 86 

...

Accuracy,▁
Validation_Loss,▁▁▄▂▄▇█▇▄▃
Validation_accuracy,▁▇▁▂▂▁▁▁█▁
epochs,▁▂▃▃▄▅▆▆▇█
training_accuracy,▁▃▅▆▅▇▇███
training_loss,█▄▃▂▂▂▂▁▁▁
Accuracy,10.25513
Validation_Loss,8.10957
Validation_accuracy,10.25513
epochs,10
training_accuracy,21.2375


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: ssqhyesh with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: Yes
wandb: 	batch_size: 32
wandb: 	data_augmentation: Yes
wandb: 	dense_sizes: 16
wandb: 	dropout: 0.2
wandb: 	epochs: 10
wandb: 	filter_size: 3
wandb: 	kernel: 8
wandb: 	kernel_aug: same
wandb: 	learning_rate: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


252
248
244
240
236
Computation ::  158281794


1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 | 27 | 28 | 29 | 30 | 
31 | 32 | 33 | 34 | 35 | 36 | 37 | 38 | 39 | 40 | 41 | 42 | 43 | 44 | 45 | 46 | 47 | 48 | 49 | 50 | 51 | 52 | 53 | 54 | 55 | 56 | 57 | 58 | 59 | 60 | 
61 | 62 | 63 | 64 | 65 | 66 | 67 | 68 | 69 | 70 | 71 | 72 | 73 | 74 | 75 | 76 | 77 | 78 | 79 | 80 | 81 | 82 | 83 | 84 | 85 | 86 | 87 | 88 | 89 | 90 | 
91 | 92 | 93 | 94 | 95 | 96 | 97 | 98 | 99 | 100 | 101 | 102 | 103 | 104 | 105 | 106 | 107 | 108 | 109 | 110 | 111 | 112 | 113 | 114 | 115 | 116 | 117 | 118 | 119 | 120 | 
121 | 122 | 123 | 124 | 125 | 126 | 127 | 128 | 129 | 130 | 131 | 132 | 133 | 134 | 135 | 136 | 137 | 138 | 139 | 140 | 141 | 142 | 143 | 144 | 145 | 146 | 147 | 148 | 149 | 150 | 
151 | 152 | 153 | 154 | 155 | 156 | 157 | 158 | 159 | 160 | 161 | 162 | 163 | 164 | 165 | 166 | 167 | 168 | 169 | 170 | 171 | 172 | 173 | 174 | 175 | 176 | 177 | 178 | 179 | 180 | 
181 | 182 | 183 | 184 